In [34]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

# To plot
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "project_energy_stat"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore  warnings 
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

import pandas as pd


Units

From Terajoules to GWH

From 'Metric tons,  thousand' to 'Metric tons'

In [35]:
df = pd.read_csv('all_energy_statistics.csv')
units=df['unit'].unique()
units

array(['Metric tons,  thousand', 'Terajoules', 'Kilowatts,  thousand',
       'Kilowatt-hours, million', 'Cubic metres, thousand', 'Metric Tons'],
      dtype=object)

In [36]:
# from metric tons thousand to metric tons
df.loc[df['unit']=='Metric tons,  thousand', 'quantity']= df['quantity']/1000
df=df.replace('Metric tons,  thousand','Metric Tons')

#From Terajoules to KWH
df.loc[df['unit']=='Terajoules', 'quantity']= df['quantity']*0.277778
df=df.replace('Terajoules','Kilowatt-hours, million')
units=df['unit'].unique()
print(units)

['Metric Tons' 'Kilowatt-hours, million' 'Kilowatts,  thousand'
 'Cubic metres, thousand']


In [37]:
len(units)

4

We have mass, energy, power and volume. Let see what categories we have in each unit

In [38]:
#Mass
cat_mass=df[df['unit']==units[0]]
mass=cat_mass['category'].unique()
mass

array(['additives_and_oxygenates', 'anthracite', 'aviation_gasoline',
       'bagasse', 'biodiesel', 'biogasoline', 'bitumen',
       'brown_coal_briquettes', 'brown_coal', 'charcoal', 'coal_tar',
       'coking_coal', 'conventional_crude_oil', 'ethane', 'fuel_oil',
       'gas_coke', 'gas_oil_diesel_oil', 'gasoline_type_jet_fuel',
       'hard_coal', 'kerosene_type_jet_fuel', 'lignite',
       'liquified_petroleum_gas', 'lubricants', 'motor_gasoline',
       'naphtha', 'natural_gas_liquids', 'of_which_biodiesel',
       'of_which_biogasoline', 'oil_shale_oil_sands',
       'other_bituminous_coal', 'other_coal_products',
       'other_hydrocarbons', 'other_kerosene', 'other_liquid_biofuels',
       'other_oil_products_n_e_c', 'paraffin_waxes', 'patent_fuel',
       'peat', 'peat_products', 'petroleum_coke', 'refinery_feedstocks',
       'refinery_gas', 'sub_bituminous_coal', 'total_refinery_output',
       'uranium',
       'white_spirit_and_special_boiling_point_industrial_spirits'],


In [39]:
#Energy
cat_energy=df[df['unit']==units[1]]
energy=cat_energy['category'].unique()
energy

array(['animal_waste', 'biogases', 'black_liquor', 'blast_furnace_gas',
       'coke_oven_coke', 'direct_use_of_geothermal_heat',
       'direct_use_of_solar_thermal_heat', 'falling_water',
       'gasworks_gas', 'geothermal', 'heat', 'hydro', 'industrial_waste',
       'municipal_wastes', 'natural_gas_including_lng',
       'nuclear_electricity', 'other_recovered_gases',
       'other_vegetal_material_and_residues', 'solar_electricity',
       'thermal_electricity', 'tide_wave_and_ocean_electricity',
       'total_electricity', 'wind_electricity'], dtype=object)

In [40]:
#Power
cat_power=df[df['unit']==units[2]]
cat_power['category'].unique()

array(['electricity_net_installed_capacity_of_electric_power_plants'],
      dtype=object)

In [41]:
cat_volume=df[df['unit']==units[3]]
cat_volume['category'].unique()

array(['fuelwood'], dtype=object)

We have 46 categories for mass, 23 for energy, one for power and one for volume

Classify by 9 Categories:

-Solid
-Oil
-gaseous
-Electricity
-heat
-Renewable
-Uranium



In [42]:

df.head()


,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,Metric Tons,0.005,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,Metric Tons,0.017,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,Metric Tons,0.000,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,Metric Tons,0.000,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,Metric Tons,0.035,NaN,additives_and_oxygenates


In [43]:
# dictionary to classify each one particular enrgy into 9 main categories
dic=pd.read_csv('dicionary.csv',encoding='cp1252')
dic.head(10)

,Annex I – SIEC and Questionnaire product codes,Unnamed: 1,Unnamed: 2,a
0,Table A.1: Correspondence between SIEC codes a...,NaN,NaN,CL01 Production
1,0 Coal,NaN,NaN,CL022 Receipts from other sources
2,01 Hard coal CL,NaN,NaN,CL03 Imports
3,011 0110 Anthracite AT,NaN,NaN,Product code CL04 Exports
4,012 Bituminous coal,NaN,NaN,CL051 International marine bunkers
5,0121 Coking coal CC,NaN,NaN,Flow code CL06 Stock changes
6,0129 Other bituminous coal OB,NaN,NaN,CLGA Total energy supply
7,02 Brown coal LB,NaN,NaN,CLSD Statistical differences
8,021 0210 Sub-bituminous coal SB,NaN,NaN,CL08 Transformation
9,022 0220 Lignite LN,NaN,NaN,"CL088 Transformation in electricity, CHP and h..."


In [44]:
dic.columns=['category','Number_Cat','flow']
dic.head(2)

,category,Number_Cat,Name_cat,flow
0,Table A.1: Correspondence between SIEC codes a...,NaN,NaN,CL01 Production
1,0 Coal,NaN,NaN,CL022 Receipts from other sources


In [45]:
#NumberCode is the first number in category
dic['Number_Cat']=dic.category.str.extract('(\d)')
#eliminate any numbers from NameCode
dic['Name_Cat']=dic.category.replace(regex=[r'(\d)'],value=[''])
#Eliminate the first space on Name_cat
dic['Name_Cat']=dic.Name_Cat.replace(['(^\s+)'],[''],regex=True)
#Replace spaces in the middle by '_'
dic['Name_Cat']=dic.Name_Cat.replace(['(\s+)'],['_'],regex=True)
#Convert all carachters to lower
dic['Name_Cat']=dic['Name_Cat'].str.lower()
dic['Number_Cat'][0]='-'
dic['Name_Cat'][0]='-'
dic.head()


<ipython-input-45-80afa10c56c5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dic['Number_Cat'][0]='-'
<ipython-input-45-80afa10c56c5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dic['Name_Cat'][0]='-'


,category,Number_Cat,Name_cat,flow,Name_Cat
0,Table A.1: Correspondence between SIEC codes a...,-,NaN,CL01 Production,-
1,0 Coal,0,NaN,CL022 Receipts from other sources,coal
2,01 Hard coal CL,0,NaN,CL03 Imports,hard_coal_cl
3,011 0110 Anthracite AT,0,NaN,Product code CL04 Exports,anthracite_at
4,012 Bituminous coal,0,NaN,CL051 International marine bunkers,bituminous_coal


In [46]:
Unique_Names = dic['Name_Cat'].unique()
Unique_codes = dic['Number_Cat'].unique()
Unique_category = dic['category'].unique()
len(Unique_Names),len(Unique_codes),len(Unique_category)

(110, 12, 117)

In [47]:
dic

,category,Number_Cat,Name_cat,flow,Name_Cat
0,Table A.1: Correspondence between SIEC codes a...,-,NaN,CL01 Production,-
1,0 Coal,0,NaN,CL022 Receipts from other sources,coal
2,01 Hard coal CL,0,NaN,CL03 Imports,hard_coal_cl
3,011 0110 Anthracite AT,0,NaN,Product code CL04 Exports,anthracite_at
4,012 Bituminous coal,0,NaN,CL051 International marine bunkers,bituminous_coal
...,...,...,...,...,...
112,Uranium for this questionnaire (included under...,9,NaN,NaN,uranium_for_this_questionnaire_(included_under...
113,92 Other nuclear fuels,9,NaN,NaN,other_nuclear_fuels
114,920 9200 Other nuclear fuels,9,NaN,NaN,other_nuclear_fuels
115,99 Other fuels n.e.c.,9,NaN,NaN,other_fuels_n.e.c.


In [48]:

manual_cat=[] #Categories in dic but function contains doesn't spot them
auto_cat=[]
c=df['category'].unique()
for i in range (1,len(c)):
    a=dic['Name_Cat'].str.contains(c[i]).any()# See if the dictionary contains category i 
    if a:
        dic.loc[dic['Name_Cat'].str.contains(c[i]),'Name_final_cat']=c[i]
        auto_cat.append(c[i])
        
    else:
        manual_cat.append(c[i])

In [49]:
len(auto_cat), len(c)

(46, 71)

In [51]:
#Fill dataframe with numbers of each category
first_row= dic_and_df = 0
for i in range (0,len(auto_cat)):
    dic_and_df = dic['Number_Cat'][dic['Name_final_cat']==auto_cat[i]]
    if dic_and_df.any():
        first_row = int(dic_and_df.iloc[0]) # choosing only first row
        df.loc[df['category']==auto_cat[i],'Number_Cat'] = first_row 


In [52]:
#completing Manually categories  plus 'brown_coal_briquettes', 'hydro' as they were missing
manually_completed_cat = pd.DataFrame({'Name_category': ['additives_and_oxygenates','sub_bituminous_coal', 'natural_gas_including_lng','direct_use_of_geothermal_heat',
                'electricity_net_installed_capacity_of_electric_power_plants','falling_water','geothermal',
                'direct_use_of_solar_thermal_heat','gas_oil_diesel_oil','gasoline_type_jet_fuel','gasworks_gas','kerosene_type_jet_fuel',
                'liquified_petroleum_gas','municipal_wastes','nuclear_electricity','of_which_biodiesel','of_which_biogasoline', 'oil_shale_oil_sands',
                'other_oil_products_n_e_c','solar_electricity', 'thermal_electricity', 'tide_wave_and_ocean_electricity', 'total_electricity',
                'wind_electricity','total_refinery_output' ,'brown_coal_briquettes', 'hydro'],
                'Number_category': ['4','2', '3','8','7','7','8','8','4','4','3','4','4','6','7','5','5','2', '4','7','7','7','7','7','4','7','0']})

In [53]:
len(manually_completed_cat) + len(auto_cat)

73

In [54]:
# Fill dataframe with the rest of categories
for i in range (0, manually_completed_cat.shape[0]):
    Name_category = manually_completed_cat['Name_category'][i]
    Number_category = manually_completed_cat['Number_category'][i]
    df.loc[df['category']==Name_category,'Number_Cat']=int(Number_category)



In [55]:
df.Number_Cat.unique()

array([4., 5., 0., 7., 8., 3., 6., 2., 1., 9.])

In [56]:
missing = df[df.Number_Cat.isnull()]['category'].unique()
missing

array([], dtype=object)